In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob

In [2]:
# Must include MGRA
columns_to_check = ['mgra', 'pop', 'hhp', 'gq_civ', 'gq_mil']

In [3]:
config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
DS35_datafiles = ['DS35_2016', 'DS35_2018', 'DS35_2020', 'DS35_2023', 'DS35_2025', 'DS35_2026', 'DS35_2029', 'DS35_2030', 'DS35_2032', 'DS35_2035', 'DS35_2040', 'DS35_2045', 'DS35_2050']

In [6]:
DS41_datafiles = ['DS41_2016', 'DS41_2018', 'DS41_2020', 'DS41_2023', 'DS41_2025', 'DS41_2026', 'DS41_2029', 'DS41_2030', 'DS41_2032', 'DS41_2035', 'DS41_2040', 'DS41_2045', 'DS41_2050']

In [7]:
older_DS = 'DS35'
current_DS = 'DS41'

# Creating the MGRA level Data Frame

In [8]:
# Older Data Frame
DS35_Data = pd.DataFrame(columns = columns_to_check)
for file_name in DS35_datafiles:
    working_df = pd.read_csv(config[older_DS][file_name], usecols=columns_to_check)
    working_df['year'] = f"{file_name[-4:]}"
    DS35_Data = DS35_Data.append(working_df)

In [9]:
#Making it original 
DS35_Data.columns = [x + '_DS35' for x in DS35_Data.columns]

In [10]:
# Newer Data Frame
DS41_Data = pd.DataFrame(columns = columns_to_check)
for file_name in DS41_datafiles:
    working_df = pd.read_csv(config[current_DS][file_name], usecols=columns_to_check)
    working_df['year'] = f"{file_name[-4:]}"
    DS41_Data = DS41_Data.append(working_df)

In [11]:
# making it original
DS41_Data.columns = [x + '_DS41' for x in DS41_Data.columns]

In [12]:
# Combine to make one Data Frame
mgra_df = pd.concat([DS35_Data, DS41_Data], axis=1)

In [13]:
# Clean up MGRA DF
mgra_df = mgra_df.rename(columns={'mgra_DS35':'mgra'})
mgra_df = mgra_df.drop(['mgra_DS41', 'year_DS35'], axis=1)
mgra_df = mgra_df.groupby(['mgra', 'year_DS41']).sum()

In [14]:
mgra_df

pop_DS35  hhp_DS35  gq_civ_DS35  gq_mil_DS35  pop_DS41  \
mgra  year_DS41                                                           
1     2016             41        41            0            0        41   
      2018             53        53            0            0        50   
      2020             49        49            0            0        43   
      2023             47        47            0            0        41   
      2025             40        40            0            0        42   
...                   ...       ...          ...          ...       ...   
23002 2032            253       253            0            0       232   
      2035            265       265            0            0       239   
      2040            268       268            0            0       256   
      2045            260       260            0            0       263   
      2050            245       245            0            0       268   

                 hhp_DS41  gq_civ_DS41  gq_mil_DS41  
mgra  year_DS41                                      
1     2016             41            0            0  
      2018             50            0            0  
      2020             43            0            0  
      2023             41            0            0  
      2025             42            0            0  
...                   ...          ...          ...  
23002 2032            232            0            0  
      2035            239            0            0  
      2040            256            0            0  
      2045            263            0            0  
      2050            268            0            0  

[299026 rows x 8 columns]

In [88]:
mgra_df.isnull().sum()

pop_DS35       0
hhp_DS35       0
gq_civ_DS35    0
gq_mil_DS35    0
pop_DS41       0
hhp_DS41       0
gq_civ_DS41    0
gq_mil_DS41    0
dtype: int64

In [15]:
hhp_diff = (mgra_df['hhp_DS41'] - mgra_df['hhp_DS35']).to_frame(name='hhp_diff')
large_diff_hhp = hhp_diff[(hhp_diff['hhp_diff'] > 500) | (hhp_diff['hhp_diff'] < -500)]

# Grabbing The Data From SQL

In [16]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [17]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [18]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [19]:
sql_df_all

,mgra,geotype,geozone
0,614,cpa,College Area
1,614,jurisdiction,San Diego
2,614,cpa,Mid-City:Kensington-Talmadge
3,614,jurisdiction,San Diego
4,615,cpa,Mid-City:Kensington-Talmadge
...,...,...,...
54627,2933,jurisdiction,San Diego
54628,2934,cpa,Uptown
54629,2934,jurisdiction,San Diego
54630,2935,cpa,Uptown


# Creating the CPA DF

In [20]:
SQL_CPA = sql_df_all[sql_df_all['geotype']=='cpa']
SQL_CPA = SQL_CPA.rename(columns={'geozone':'cpa_geozone'})
SQL_CPA = SQL_CPA.drop(['geotype'], axis = 1)

In [21]:
#DS35
DS35_Data_cpa = DS35_Data.merge(SQL_CPA, how='left', left_on='mgra_DS35', right_on='mgra')
DS35_Data_cpa = DS35_Data_cpa.drop(['mgra'], axis=1)
DS35_Data_cpa = DS35_Data_cpa.rename(columns={'cpa_geozone':'cpa_geozone_DS35'})

In [22]:
# DS41
DS41_Data_cpa = DS41_Data.merge(SQL_CPA, how='left', left_on='mgra_DS41', right_on='mgra')
DS41_Data_cpa = DS41_Data_cpa.drop(['mgra'], axis=1)
#DS41_Data_cpa.columns = [x + '_DS41' for x in DS41_Data_cpa.columns]

In [23]:
# Combine to make one Data Frame
cpa_df = pd.concat([DS35_Data_cpa, DS41_Data_cpa], axis=1, join='inner')


In [24]:
# Clean up CPA DF
cpa_df = cpa_df.drop(['mgra_DS35', 'cpa_geozone_DS35', 'year_DS35', 'mgra_DS41'], axis=1)
cpa_df = cpa_df.groupby(['cpa_geozone', 'year_DS41']).sum()
cpa_df = cpa_df.drop('*Not in a CPA*') #Per meeting with Purva and Mike I was instructed that I can just drop the MGRAs that do not lie in a cpa

In [25]:
# Final CPA DF
cpa_df.head(50)

pop_DS35  hhp_DS35  gq_civ_DS35  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8513       584            0   
                          2020           9992       553            0   
                          2023          12272       568            0   
                          2025          12998       539            0   
                          2026          13002       543            0   
                          2029          13011       552            0   
                          2030          13016       557            0   
                          2032          13001       542            0   
                          2035          12977       518            0   
                          2040          12946       487            0   
                          2045          12961       502            0   
                          2050          12940       481            0   
Alpine                    2016          20788     20533          255   
                          2018          20929     20644          285   
                          2020          20495     20210          285   
                          2023          20066     19781          285   
                          2025          19838     19553          285   
                          2026          19695     19410          285   
                          2029          19312     19027          285   
                          2030          19258     18973          285   
                          2032          19090     18805          285   
                          2035          18926     18641          285   
                          2040          19081     18796          285   
                          2045          19316     19031          285   
                          2050          19522     19237          285   
Balboa Park               2016           6296      5958          182   
                          2018           6788      6294          192   
                          2020           6918      6424          192   
                          2023           6932      6438          192   
                          2025           7151      6657          192   
                          2026           7252      6758          192   
                          2029           7367      6873          192   
                          2030           7488      6994          192   
                          2032           7625      7131          192   
                          2035           7706      7212          192   
                          2040           7893      7399          192   
                          2045           8055      7561          192   
                          2050           8212      7718          192   
Barona                    2016           2667      2666            1   
                          2018           2598      2597            1   
                          2020           2587      2586            1   
                          2023           2587      2586            1   
                          2025           2571      2570            1   
                          2026           2578      2577            1   
                          2029           2503      2502            1   
                          2030           2495      2494            1   
                          2032           2487      2486            1   
                          2035           2489      2488            1   
                          2040           2626      2625            1   

                                     gq_mil_DS35  pop_DS41  hhp_DS41  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8492       563   
    

# Creating the Jurisdiction DF

In [26]:
SQL_jur = sql_df_all[sql_df_all['geotype']=='jurisdiction']
SQL_jur = SQL_jur.rename(columns={'geozone':'jur_geozone'})
SQL_jur = SQL_jur.drop(['geotype'], axis = 1)

In [27]:
#DS35
DS35_Data_jur = DS35_Data.merge(SQL_jur, how='left', left_on='mgra_DS35', right_on='mgra')
DS35_Data_jur = DS35_Data_jur.drop(['mgra'], axis=1)
DS35_Data_jur = DS35_Data_jur.rename(columns={'jur_geozone':'jur_geozone_DS35'})

In [28]:
#DS41
DS41_Data_jur = DS41_Data.merge(SQL_jur, how='left', left_on='mgra_DS41', right_on='mgra')
DS41_Data_jur = DS41_Data_jur.drop(['mgra'], axis=1)

In [29]:
jur_df = pd.concat([DS35_Data_jur, DS41_Data_jur], axis=1, join='inner')

In [30]:
# Ensure JurZones Are Equal, this should be zero as it is counting the number of falses  
sum(jur_df['jur_geozone_DS35'] == jur_df['jur_geozone']) - len(jur_df)

0

In [31]:
# Clean up JUR DF
jur_df = jur_df.drop(['mgra_DS35', 'jur_geozone_DS35', 'year_DS35', 'mgra_DS41'], axis=1)
jur_df = jur_df.groupby(['jur_geozone', 'year_DS41']).sum()

In [32]:
# Final Jurisdiction DF
jur_df.head(30)

pop_DS35  hhp_DS35  gq_civ_DS35  gq_mil_DS35  pop_DS41  \
jur_geozone year_DS41                                                           
Carlsbad    2016         125491    124576          915            0    125491   
            2018         128578    127663          915            0    128523   
            2020         130398    129483          915            0    130443   
            2023         130414    129499          915            0    130391   
            2025         130603    129688          915            0    130465   
            2026         130658    129743          915            0    130441   
            2029         130680    129765          915            0    130422   
            2030         130578    129663          915            0    130364   
            2032         130533    129618          915            0    130513   
            2035         130859    129944          915            0    130977   
            2040         131547    130632          915            0    131617   
            2045         131995    131080          915            0    131933   
            2050         132311    131396          915            0    132476   
Chula Vista 2016         272502    270799         1703            0    272502   
            2018         274735    273015         1720            0    274619   
            2020         275813    274093         1720            0    275765   
            2023         280646    278926         1720            0    280643   
            2025         283200    281480         1720            0    283106   
            2026         283216    281496         1720            0    283135   
            2029         285414    283694         1720            0    285235   
            2030         286623    284903         1720            0    286478   
            2032         289753    288033         1720            0    289588   
            2035         293841    292121         1720            0    293498   
            2040         310889    309169         1720            0    310816   
            2045         331153    329433         1720            0    331011   
            2050         339410    337690         1720            0    339289   
Coronado    2016          25369     18803          149         6417     25369   
            2018          25465     18899          149         6417     25469   
            2020          25505     18939          149         6417     25478   
            2023          25577     19011          149         6417     25550   

                       hhp_DS41  gq_civ_DS41  gq_mil_DS41  
jur_geozone year_DS41                                      
Carlsbad    2016         124576          915            0  
            2018         127608          915            0  
            2020         129528          915            0  
            2023         129476          915            0  
            2025         129550          915            0  
            2026         129526          915            0  
            2029         129507          915            0  
            2030         129449          915            0  
            2032         129598          915            0  
            2035         130062          915            0  
            2040         130702          915            0  
            2045         131018          915            0  
            2050         131561          915            0  
Chula Vista 2016         270799         1703            0  
            2018         272899         1720            0  
            2020         274045         1720            0  
            2023         278923         1720            0  
            2025         281386         1720            0  
            2026         281415         1720            0  
            2029         283515         1720            0  
            2030         284758         1720            0  
            2032         287868         1720      

# Grabbing the DOF Data 

In [33]:
#dof_data = pd.read_excel(config['dof']['dof_data'])

# MGRA Analysis

In [34]:
mgra_diff_df = pd.DataFrame(index=mgra_df.index)

In [35]:
#MGRA Diff
mgra_diff_df['pop_diff'] = mgra_df['pop_DS41'] - mgra_df['pop_DS35']
mgra_diff_df['hhp_diff'] = mgra_df['hhp_DS41'] - mgra_df['hhp_DS35']
mgra_diff_df['gq_civ_diff'] = mgra_df['gq_civ_DS41'] - mgra_df['gq_civ_DS35']
mgra_diff_df['gq_mil_diff'] = mgra_df['gq_mil_DS41'] - mgra_df['gq_mil_DS35']

In [36]:
#mgra_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS3541_v2_results/mgra/DS4135_diff.csv', index = True)

In [37]:
# MGRA Stats 
mgra_stats = pd.DataFrame(columns=mgra_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in mgra_stats.columns:
    mgra_stats[col]['Max'] = np.max(list(mgra_diff_df[col]))
    mgra_stats[col]['Min'] = np.min(list(mgra_diff_df[col]))
    mgra_stats[col]['Average'] = np.mean(list(mgra_diff_df[col]))

In [38]:
mgra_stats.head()


,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,831,831,0,0
Min,-326,-326,0,0
Average,0.0,0.0,0.0,0.0


In [39]:
#mgra_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS3541_v2_results/mgra/mgra_stats.csv', index = True)

In [40]:
# Major HHP Diffs
hhp_diff = (mgra_df['hhp_DS41'] - mgra_df['hhp_DS35']).to_frame(name='hhp_diff')
large_diff_hhp = hhp_diff[(hhp_diff['hhp_diff'] > 500) | (hhp_diff['hhp_diff'] < -500)]

In [41]:
#large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS3541_v2_results/mgra/large_diff_hhp.csv', index = True)

In [42]:
# Major Pop Diff
pop_diff = (mgra_df['pop_DS41'] - mgra_df['pop_DS35']).to_frame(name='pop_diff')
large_diff_pop = pop_diff[(pop_diff['pop_diff'] > 500) | (pop_diff['pop_diff'] < -500)]

In [43]:
#large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS3541_v2_results/mgra/large_diff_pop.csv', index = True)


# Region Analysis

In [44]:
mgra_df

pop_DS35  hhp_DS35  gq_civ_DS35  gq_mil_DS35  pop_DS41  \
mgra  year_DS41                                                           
1     2016             41        41            0            0        41   
      2018             53        53            0            0        50   
      2020             49        49            0            0        43   
      2023             47        47            0            0        41   
      2025             40        40            0            0        42   
...                   ...       ...          ...          ...       ...   
23002 2032            253       253            0            0       232   
      2035            265       265            0            0       239   
      2040            268       268            0            0       256   
      2045            260       260            0            0       263   
      2050            245       245            0            0       268   

                 hhp_DS41  gq_civ_DS41  gq_mil_DS41  
mgra  year_DS41                                      
1     2016             41            0            0  
      2018             50            0            0  
      2020             43            0            0  
      2023             41            0            0  
      2025             42            0            0  
...                   ...          ...          ...  
23002 2032            232            0            0  
      2035            239            0            0  
      2040            256            0            0  
      2045            263            0            0  
      2050            268            0            0  

[299026 rows x 8 columns]

In [45]:
# mgra_names = []
# for name in mgra_df.index:
#     mgra_names.append(name[0])
# unique_mgra_names = list(np.unique(np.array(mgra_names)))
# years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [46]:
# mgra_sums_df = pd.DataFrame(columns=mgra_df.columns, index=years)
# for col in mgra_df:
#     for year in years:
#             total = 0
#             for mgra in unique_mgra_names:
#                 total += int(mgra_df.loc[mgra].loc[[year]][col])
#             mgra_sums_df[col][year] = total

In [47]:
# mgra_sums_df

In [48]:
# mgra_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/mgra_sums.csv', index = True)

# CPA Analysis

In [49]:
cpa_df.head()

pop_DS35  hhp_DS35  gq_civ_DS35  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8513       584            0   
                          2020           9992       553            0   
                          2023          12272       568            0   
                          2025          12998       539            0   

                                     gq_mil_DS35  pop_DS41  hhp_DS41  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8492       563   
                          2020              9439     10005       566   
                          2023             11704     12255       551   
                          2025             12459     13019       560   

                                     gq_civ_DS41  gq_mil_DS41  
cpa_geozone               year_DS41                            
32nd Street Naval Station 2016                 0         7586  
                          2018                 0         7929  
                          2020                 0         9439  
                          2023                 0        11704  
                          2025                 0        12459

In [50]:
cpa_diff_df = pd.DataFrame(index=cpa_df.index)

In [51]:
#41 minus 35
cpa_diff_df['pop_diff'] = cpa_df['pop_DS41'] - cpa_df['pop_DS35']
cpa_diff_df['hhp_diff'] = cpa_df['hhp_DS41'] - cpa_df['hhp_DS35']
cpa_diff_df['gq_civ_diff'] = cpa_df['gq_civ_DS41'] - cpa_df['gq_civ_DS35']
cpa_diff_df['gq_mil_diff'] = cpa_df['gq_mil_DS41'] - cpa_df['gq_mil_DS35']

In [52]:
# cpa_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/cpa/CPA_DS4135_diff.csv', index = True)

In [53]:
cpa_diff_df.head(10)

pop_diff  hhp_diff  gq_civ_diff  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016              0         0            0   
                          2018            -21       -21            0   
                          2020             13        13            0   
                          2023            -17       -17            0   
                          2025             21        21            0   
                          2026             19        19            0   
                          2029             -2        -2            0   
                          2030             -7        -7            0   
                          2032            -10       -10            0   
                          2035            -27       -27            0   

                                     gq_mil_diff  
cpa_geozone               year_DS41               
32nd Street Naval Station 2016                 0  
                          2018                 0  
                          2020                 0  
                          2023                 0  
                          2025                 0  
                          2026                 0  
                          2029                 0  
                          2030                 0  
                          2032                 0  
                          2035                 0

In [54]:
# 41 minus 35
cpa_stats = pd.DataFrame(columns=cpa_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in mgra_stats.columns:
    cpa_stats[col]['Max'] = np.max(list(cpa_diff_df[col]))
    cpa_stats[col]['Min'] = np.min(list(cpa_diff_df[col]))
    cpa_stats[col]['Average'] = np.mean(list(cpa_diff_df[col]))

In [55]:
cpa_stats

,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,1985,1985,0,0
Min,-1112,-1112,0,0
Average,7.058355,7.058355,0.0,0.0


In [56]:
# cpa_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/cpa/CPA_DS4135_stats.csv', index = True)

## CPA Sums

In [57]:
CPA_names = []
for name in cpa_df.index:
    CPA_names.append(name[0])
unique_cpa_names = list(np.unique(np.array(CPA_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [58]:
cpa_sums_df = pd.DataFrame(columns=cpa_df.columns, index=years)
for col in cpa_df:
    for year in years:
            total = 0
            for cpa in unique_cpa_names:
                total += int(cpa_df.loc[cpa].loc[[year]][col])
            cpa_sums_df[col][year] = total


In [59]:
cpa_sums_df

,pop_DS35,hhp_DS35,gq_civ_DS35,gq_mil_DS35,pop_DS41,hhp_DS41,gq_civ_DS41,gq_mil_DS41
2016,2341482,2210596,60051,70835,2341482,2210596,60051,70835
2018,2379713,2234972,66793,77948,2379521,2234780,66793,77948
2020,2407915,2255027,70410,82478,2407938,2255050,70410,82478
2023,2456210,2294403,72534,89273,2456062,2294255,72534,89273
2025,2494735,2330497,72700,91538,2494685,2330447,72700,91538
2026,2514097,2349776,72783,91538,2514083,2349762,72783,91538
2029,2565789,2401219,73032,91538,2566309,2401739,73032,91538
2030,2582817,2418164,73115,91538,2583993,2419340,73115,91538
2032,2609492,2444673,73281,91538,2611456,2446637,73281,91538
2035,2649048,2483980,73530,91538,2651159,2486091,73530,91538


In [60]:
# cpa_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/cpa/CPA_DS4135_sums.csv', index = True)

In [61]:
cpa_df

pop_DS35  hhp_DS35  gq_civ_DS35  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8513       584            0   
                          2020           9992       553            0   
                          2023          12272       568            0   
                          2025          12998       539            0   
...                                       ...       ...          ...   
Via De La Valle           2032           1206      1206            0   
                          2035           1217      1217            0   
                          2040           1271      1271            0   
                          2045           1278      1278            0   
                          2050           1295      1295            0   

                                     gq_mil_DS35  pop_DS41  hhp_DS41  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8492       563   
                          2020              9439     10005       566   
                          2023             11704     12255       551   
                          2025             12459     13019       560   
...                                          ...       ...       ...   
Via De La Valle           2032                 0      1178      1178   
                          2035                 0      1175      1175   
                          2040                 0      1239      1239   
                          2045                 0      1303      1303   
                          2050                 0      1268      1268   

                                     gq_civ_DS41  gq_mil_DS41  
cpa_geozone               year_DS41                            
32nd Street Naval Station 2016                 0         7586  
                          2018                 0         7929  
                          2020                 0         9439  
                          2023                 0        11704  
                          2025                 0        12459  
...                                          ...          ...  
Via De La Valle           2032                 0            0  
                          2035                 0            0  
                          2040                 0            0  
                          2045                 0            0  
                          2050                 0            0  

[1131 rows x 8 columns]

In [62]:
# Major HHP Diffs
cpa_hhp_diff = (cpa_df['hhp_DS41'] - cpa_df['hhp_DS35']).to_frame(name='hhp_diff')
cpa_large_diff_hhp = cpa_hhp_diff[(cpa_hhp_diff['hhp_diff'] > 750) | (cpa_hhp_diff['hhp_diff'] < -750)]

In [63]:
# cpa_large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/cpa/CPA_DS4135_hhp_diff.csv', index = True)

In [64]:
# Major pop Diffs
cpa_pop_diff = (cpa_df['pop_DS41'] - cpa_df['pop_DS35']).to_frame(name='pop_diff')
cpa_large_diff_pop = cpa_pop_diff[(cpa_pop_diff['pop_diff'] > 500) | (cpa_pop_diff['pop_diff'] < -500)]

In [65]:
# cpa_large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/cpa/CPA_DS4135_pop_diff.csv', index = True)

# Check the Jurisdiction Level

In [66]:
jur_diff_df = pd.DataFrame(index=jur_df.index)

In [67]:
#41 minus 35
jur_diff_df['pop_diff'] = jur_df['pop_DS41'] - jur_df['pop_DS35']
jur_diff_df['hhp_diff'] = jur_df['hhp_DS41'] - jur_df['hhp_DS35']
jur_diff_df['gq_civ_diff'] = jur_df['gq_civ_DS41'] - jur_df['gq_civ_DS35']
jur_diff_df['gq_mil_diff'] = jur_df['gq_mil_DS41'] - jur_df['gq_mil_DS35']

In [68]:
jur_diff_df

pop_diff  hhp_diff  gq_civ_diff  gq_mil_diff
jur_geozone year_DS41                                              
Carlsbad    2016              0         0            0            0
            2018            -55       -55            0            0
            2020             45        45            0            0
            2023            -23       -23            0            0
            2025           -138      -138            0            0
...                         ...       ...          ...          ...
Vista       2032           -176      -176            0            0
            2035             13        13            0            0
            2040             48        48            0            0
            2045            136       136            0            0
            2050            146       146            0            0

[247 rows x 4 columns]

In [69]:
# jur_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_diff.csv', index = True)

In [70]:
# 41 minus 35
jur_stats = pd.DataFrame(columns=jur_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in jur_stats.columns:
    jur_stats[col]['Max'] = np.max(list(jur_diff_df[col]))
    jur_stats[col]['Min'] = np.min(list(jur_diff_df[col]))
    jur_stats[col]['Average'] = np.mean(list(jur_diff_df[col]))
jur_stats

,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,2409,2409,0,0
Min,-960,-960,0,0
Average,26.757085,26.757085,0.0,0.0


In [71]:
# jur_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_stats.csv', index = True)

# Checking the sums

In [72]:
jur_names = []
for name in jur_df.index:
    jur_names.append(name[0])
unique_jur_names = list(np.unique(np.array(jur_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [73]:
jur_sums_df = pd.DataFrame(columns=jur_df.columns, index=years)
for col in jur_df:
    for year in years:
            total = 0
            for jur in unique_jur_names:
                total += int(jur_df.loc[jur].loc[[year]][col])
            jur_sums_df[col][year] = total

In [74]:
jur_sums_df

,pop_DS35,hhp_DS35,gq_civ_DS35,gq_mil_DS35,pop_DS41,hhp_DS41,gq_civ_DS41,gq_mil_DS41
2016,3906368,3732674,77690,96004,3906368,3732674,77690,96004
2018,3960441,3772343,84651,103447,3959547,3771449,84651,103447
2020,4003343,3805479,88377,109487,4003080,3805216,88377,109487
2023,4076107,3866589,90971,118547,4075581,3866063,90971,118547
2025,4127648,3914538,91543,121567,4127813,3914703,91543,121567
2026,4151940,3938544,91829,121567,4151861,3938465,91829,121567
2029,4214983,4000729,92687,121567,4214434,4000180,92687,121567
2030,4235273,4020733,92973,121567,4236157,4021617,92973,121567
2032,4270172,4055466,93139,121567,4272261,4057555,93139,121567
2035,4320952,4105997,93388,121567,4323047,4108092,93388,121567


In [75]:
# jur_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_sums.csv', index = True)

In [76]:
jur_df

pop_DS35  hhp_DS35  gq_civ_DS35  gq_mil_DS35  pop_DS41  \
jur_geozone year_DS41                                                           
Carlsbad    2016         125491    124576          915            0    125491   
            2018         128578    127663          915            0    128523   
            2020         130398    129483          915            0    130443   
            2023         130414    129499          915            0    130391   
            2025         130603    129688          915            0    130465   
...                         ...       ...          ...          ...       ...   
Vista       2032         125743    123416         2327            0    125567   
            2035         126773    124446         2327            0    126786   
            2040         127775    125448         2327            0    127823   
            2045         127911    125584         2327            0    128047   
            2050         128122    125795         2327            0    128268   

                       hhp_DS41  gq_civ_DS41  gq_mil_DS41  
jur_geozone year_DS41                                      
Carlsbad    2016         124576          915            0  
            2018         127608          915            0  
            2020         129528          915            0  
            2023         129476          915            0  
            2025         129550          915            0  
...                         ...          ...          ...  
Vista       2032         123240         2327            0  
            2035         124459         2327            0  
            2040         125496         2327            0  
            2045         125720         2327            0  
            2050         125941         2327            0  

[247 rows x 8 columns]

In [77]:
# Major pop Diffs
jur_pop_diff = (jur_df['pop_DS41'] - jur_df['pop_DS35']).to_frame(name='pop_diff')
jur_large_diff_pop = jur_pop_diff[(jur_pop_diff['pop_diff'] > 500) | (jur_pop_diff['pop_diff'] < -500)]

In [78]:
jur_large_diff_pop

pop_diff
jur_geozone    year_DS41          
Escondido      2025            561
San Diego      2029           1103
               2030           1557
               2032           1941
               2035           2409
               2040           2402
               2045           1461
               2050           1286
Unincorporated 2029           -583
               2040           -728
               2045           -960
               2050           -868

In [79]:
# jur_large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_pop_diff.csv', index = True)

In [80]:
# Major hhp Diffs
jur_hhp_diff = (jur_df['hhp_DS41'] - jur_df['hhp_DS35']).to_frame(name='hhp_diff')
jur_large_diff_hhp = jur_hhp_diff[(jur_hhp_diff['hhp_diff'] > 750) | (jur_hhp_diff['hhp_diff'] < -750)]

In [81]:
jur_large_diff_hhp

hhp_diff
jur_geozone    year_DS41          
San Diego      2029           1103
               2030           1557
               2032           1941
               2035           2409
               2040           2402
               2045           1461
               2050           1286
Unincorporated 2045           -960
               2050           -868

In [82]:
# jur_large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_hhp_diff.csv', index = True)

In [83]:
jur_names = []
for name in jur_df.index:
    jur_names.append(name[0])
unique_jur_names = list(np.unique(np.array(jur_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [84]:
pct_diff_df_jur = pd.DataFrame(columns=jur_df.columns, index=jur_df.index)
# Year over year increase 
for zone_name in unique_jur_names:
    for i in range(1,len(years)):
        for col in jur_df.columns:
            current_val = jur_df.loc[zone_name].loc[years[i]][col]
            prior_val = jur_df.loc[zone_name].loc[years[i-1]][col]
            percent_diff = ((current_val - prior_val)/prior_val)*100
            pct_diff_df_jur.loc[zone_name].loc[years[i]][col] = percent_diff
pct_diff_df_jur
            



C:\Users\cra\AppData\Local\Temp/ipykernel_11408/2621092274.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  percent_diff = ((current_val - prior_val)/prior_val)*100


pop_DS35  hhp_DS35 gq_civ_DS35 gq_mil_DS35  pop_DS41  \
jur_geozone year_DS41                                                         
Carlsbad    2016            NaN       NaN         NaN         NaN       NaN   
            2018       2.459937  2.478005         0.0         NaN   2.41611   
            2020       1.415483  1.425628         0.0         NaN  1.493896   
            2023        0.01227  0.012357         0.0         NaN -0.039864   
            2025       0.144923  0.145947         0.0         NaN  0.056752   
...                         ...       ...         ...         ...       ...   
Vista       2032         0.4923  0.501629         0.0         NaN  0.538857   
            2035       0.819131  0.834576         0.0         NaN  0.970796   
            2040       0.790389  0.805169         0.0         NaN  0.817914   
            2045       0.106437  0.108411         0.0         NaN  0.175242   
            2050       0.164958  0.168015         0.0         NaN  0.172593   

                       hhp_DS41 gq_civ_DS41 gq_mil_DS41  
jur_geozone year_DS41                                    
Carlsbad    2016            NaN         NaN         NaN  
            2018       2.433856         0.0         NaN  
            2020       1.504608         0.0         NaN  
            2023      -0.040146         0.0         NaN  
            2025       0.057153         0.0         NaN  
...                         ...         ...         ...  
Vista       2032       0.549087         0.0         NaN  
            2035       0.989127         0.0         NaN  
            2040       0.833206         0.0         NaN  
            2045       0.178492         0.0         NaN  
            2050       0.175787         0.0         NaN  

[247 rows x 8 columns]

In [85]:
Maximum_percent_difference = pd.DataFrame(pct_diff_df_jur.max())
Minimum_percent_difference = pd.DataFrame(pct_diff_df_jur.min())

In [86]:
# Maximum_percent_difference.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_max_diff.csv', index = True)

In [87]:
# Minimum_percent_difference.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4135_results/jur/jur_DS4135_min_diff.csv', index = True)